In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import EdgeWeightNorm, GraphConv,RelGraphConv, SAGEConv
from dgl.data import DGLDataset
import mysql.connector
import dgl.data
import numpy as np
import pandas as pd
import os
from dgl import save_graphs, load_graphs
import matplotlib.pyplot as plt
import random
from Datasets import PaperDataset
from Models import GraphClassificationModelCrossEntropy, GraphClassificationModelBinaryCrossEntropy
import networkx as nx
import scipy.sparse as sp
import itertools
import torch as th
import dgl.function as fn
from sklearn.metrics import roc_auc_score
import dgl.nn as dglnn
import random
from torchmetrics.classification import BinaryAUROC
from torch.utils.data import TensorDataset, DataLoader

# Connect to database

In [4]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [5]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)
in_params_test = ','.join(['%s'] * len(test_papers))

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)

in_params_val = ','.join(['%s'] * len(val_papers))
in_params = ','.join(['%s'] * len(train_papers + test_papers))

## Fetch Data primary

In [6]:
def remap(x, mapping):
    return mapping.get(x, x)

In [7]:
cursor.execute("select b.ReferenceID, b.ReferencedByID from referencedBy b, Papers p where b.ReferencedByID in (%s) and b.ReferenceID = p.PaperID and p.`primary author` in (select AuthorID from affiliatedTo)" % in_params, total_papers)
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

all_papers_list = all_papers.tolist()
all_params = ','.join(['%s'] * len(all_papers_list))

cursor.execute("select p.PaperID, p.`primary author`, a.Gender from Papers p, Authors a where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = a.AuthorID" % all_params, all_papers_list)
author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [8]:
cursor.execute("select distinct p.`primary author` from Papers p where p.PaperID in (%s)" % in_params, total_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()

## Fetch the Gender

In [9]:
all_params_gender = ','.join(['%s'] * len(all_authors))

In [10]:
cursor.execute("select distinct AuthorID, Gender from Authors a where a.AuthorID in (%s)" % all_params_gender, all_authors.tolist())
gender_edges = np.array(cursor.fetchall())

In [11]:
gender_edges[:,0] = vec_remap(gender_edges[:,0], remapped_authors)

## Fetch the affiliation

In [12]:
cursor.execute("select p.PaperID, p.`primary author`, b.AffiliationID, b.Country from Papers p, affiliatedTo af, Affiliations b where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = af.AuthorID and af.affiliatedTo = b.AffiliationID" % all_params, all_papers_list)
author_info = np.array(cursor.fetchall())

all_affiliations = np.unique(author_info[:,2])
remapped_affiliations = {all_affiliations[i]: i  for i in range(len(all_affiliations))}

all_countries = np.unique(author_info[:,3])
remapped_countries = {all_countries[i]: i  for i in range(len(all_countries))}

author_info[:,2] = vec_remap(author_info[:,2], remapped_affiliations)
author_info[:,3] = vec_remap(author_info[:,3], remapped_countries)
author_info[:,1] = vec_remap(author_info[:,1], remapped_authors)
author_info[:,0] = vec_remap(author_info[:,0], remapped_papers)

affiliation_edges = author_info[:,[2,1]]
affiliation_edges = np.unique(affiliation_edges, axis=0)

country_edges = author_info[:,[3,2]]
country_edges = np.unique(country_edges, axis=0)

# Fetch data

In [5]:
cursor.execute("select * from referencedBy b where b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)")
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)
remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

cursor.execute("select * from authoredBy where PaperID in (select PaperID from Papers) and (PaperID in (select ReferenceID from referencedBy) or PaperID in (select ReferencedByID from referencedBy)) and PaperID in (select p.PaperID from Papers p where p.Leaf = FALSE) and AuthoredByID in (select a.AuthorID from affiliatedTo a)")

author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [21]:
cursor.execute("select distinct b.AuthoredByID from Papers p, authoredBy b where p.Leaf = False and p.PaperID in (%s) and p.PaperID = b.PaperID and b.AuthoredByID in (select a.AuthorID from affiliatedTo a)" % in_params, train_papers+test_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()

# Paper data

Since we use only paper information, we remove all other "authoredBy" edges

In [16]:
test_papers = [x for x in test_papers if x not in val_papers]

In [20]:
remapped_test_papers = th.tensor([remapped_papers[idx] for idx in test_papers])
remapped_val_papers = th.tensor([remapped_papers[idx] for idx in val_papers])
remapped_train_papers = th.tensor([remapped_papers[idx] for idx in train_papers])
remapped_authors_in_question = th.tensor([remapped_authors[idx] for idx in authors_in_question])

In [21]:
hetero_graph = dgl.heterograph(
    {
        ('paper', 'authored', 'author'): (author_edges[:,0].astype(int), author_edges[:,1].astype(int)),
        ('author', "writes", "paper"): (author_edges[:,1].astype(int), author_edges[:,0].astype(int)),
        ('paper', 'cites', 'paper'): (paper_edges[:,0].astype(int), paper_edges[:,1].astype(int)),
        ('gender', 'gendered', 'author'): (gender_edges[:,1].astype(int), gender_edges[:,0].astype(int)),
        ('paper', 'authored_gender', 'gender'): (author_edges[:,0].astype(int), author_edges[:,2].astype(int)),
        ('paper', 'authored_affiliation', 'affiliation'): (author_info[:,0].astype(int), author_info[:,2].astype(int)),
        ('affiliation', 'affiliated', 'author'): (affiliation_edges[:,0].astype(int), affiliation_edges[:,1].astype(int)),
        ('paper', 'authored_country', 'country'): (author_info[:,0].astype(int), author_info[:,3].astype(int)),
        ('country', 'contains', 'affiliation'): (country_edges[:,0].astype(int), country_edges[:,1].astype(int))
    }
)
hetero_graph.nodes['author'].data['feature'] = hetero_graph.nodes('author')
hetero_graph.nodes['paper'].data['feature'] = hetero_graph.nodes('paper')

In [22]:
hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 36167, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [71]:
"""
edges = hetero_graph.edges(etype="authored")
author_mask = ~torch.isin(edges[1], remapped_authors_in_question)
paper_edges = edges[0][author_mask]
author_edges = edges[1][author_mask]
author_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")
hetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")
"""

'\nedges = hetero_graph.edges(etype="authored")\nauthor_mask = ~torch.isin(edges[1], remapped_authors_in_question)\npaper_edges = edges[0][author_mask]\nauthor_edges = edges[1][author_mask]\nauthor_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")\nhetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")\n'

### Split into Training, Validation, Test set

#### Train set

In [23]:
edges = hetero_graph.edges(etype="authored")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = hetero_graph.edge_ids(train_src, train_dst, etype="authored")
train_hetero_graph = dgl.remove_edges(hetero_graph, train_ids, etype="authored")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [24]:
edges = train_hetero_graph.edges(etype="authored_gender")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_gender")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_gender")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [25]:
edges = train_hetero_graph.edges(etype="authored_affiliation")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_affiliation")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_affiliation")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 2385, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [32]:
edges = train_hetero_graph.edges(etype="authored_country")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_country")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_country")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 2385, ('paper', 'authored_country', 'country'): 2385, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

# Validation Set

In [33]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
val_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [34]:
edges = val_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_gender")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [35]:
edges = val_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_affiliation")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [40]:
edges = val_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_country")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 300, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

## Test set

In [41]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
test_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 706, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [43]:
edges = test_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_gender")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 706, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 706, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [44]:
edges = test_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_affiliation")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 706, ('paper', 'authored_affiliation', 'affiliation'): 706, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 706, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [51]:
edges = test_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_country")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18355, ('paper', 'authored', 'author'): 706, ('paper', 'authored_affiliation', 'affiliation'): 706, ('paper', 'authored_country', 'country'): 706, ('paper', 'authored_gender', 'gender'): 706, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [52]:
save_graphs("./graphs/hetero_graphs_primary_all_features_w_test.bin", [train_hetero_graph, val_hetero_graph, test_hetero_graph])